In [4]:
# Listado de bibliotecas necesarias:

import pandas as pd  
import numpy as np
import re
import random
from unidecode import unidecode 

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from time import sleep  

# Si no deja importar alguna de las librerías, probar a ejecutar el comando pertinente en la terminal:
#----- pip install pandas
#----- pip install numpy
#----- pip install regex
#----- pip install unidecode
#----- pip install selenium
#----- pip install webdriver-manager

In [5]:
class Obtencion_detalles_IMDb:
    
    """Clase para extraer y guardar detalles de películas desde IMDb."""
    
    
    def __init__ (self , archivo_csv = 'Ruta o nombre del archivo.csv', columna1 = 'Nombre columna títulos', columna2 = 'Nombre columna años' , columna3 = "Nombre columna ID peli" , salida_detalles = "Ruta o nombre de salida_detalles.csv" , salida_actores = "Ruta o nombre de salida_actores.csv"):
        
        """
        Inicializa la clase Detalles_IMDb.

        Args:
            archivo_csv (str): Ruta o nombre del archivo CSV que contiene los datos de las películas.
            columna1 (str): Nombre de la columna que contiene los títulos de las películas en el archivo CSV.
            columna2 (str): Nombre de la columna que contiene los años de las películas en el archivo CSV.
            columna3 (str): Nombre de la columna que contiene los ID de las películas en el archivo CSV.
            salida_detalles (str): Ruta o nombre del archivo de salida para los detalles de las películas.
            salida_actores (str): Ruta o nombre del archivo de salida para los actores de las películas.
        """
                
        self.archivo_csv = archivo_csv
        self.columna1 = columna1
        self.columna2 = columna2
        self.columna3 = columna3
        self.salida_detalles = salida_detalles
        self.salida_actores = salida_actores
        
        
    def _preparacion (self):
        
        """Abre el navegador, se dirige a la página web, acepta cookies y se prepara para extraer los datos."""

        # Carga el archivo CSV
        self.peliculas_df = pd.read_csv(self.archivo_csv) 
        
        # Inicia el navegador Chrome en español
        self.options = webdriver.ChromeOptions()
        self.options.add_argument("--accept-lang=es")
        self.driver = webdriver.Chrome(options=self.options)

        # Abre la página de IMDb
        self.driver.get("https://www.imdb.com/")
        self.driver.maximize_window()
        
        sleep(random.uniform(1.5 , 2.0))
        
        # Acepta las cookies si es necesario
        try:
            self.driver.find_element("css selector" , "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW").click()
        except:
            pass
        
        self.info_extraida = {}
        self.lista_actores = []
        self.ultimo_descanso_corto = 0
        self.ultimo_descanso_largo = 0
        
        print(f"-- Empieza la extracción de datos")
        print(f" ")
        
                
    def _busqueda_id (self):
        
        """Realiza la búsqueda del ID de la película en IMDb."""
        
        self.nombre_peli = self.pelicula[self.columna1].lower()  # Obtiene el nombre de la película
        self.anio_peli = int(self.pelicula[self.columna2]) # Obtiene el año de la película
        self.id_peli = self.pelicula[self.columna3] # Obtiene el ID de la película
        self.reparto_encontrado = False
        
        
        self.driver.get(f"https://www.imdb.com/title/{self.id_peli}")
        
        sleep(random.uniform(1.0 , 1.5)) 
          
    
    def _obtencion_info (self):
        
        """Obtiene la información de la película y de su reparto desde IMDb."""

        try: 
            self.info = self.driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text 
                
        except: 
            
            try:
                self.driver.get(f"https://www.imdb.com/title/{self.id_peli}")
                sleep(random.uniform(1.0 , 1.5))
                self.info = self.driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text 
                print(f"Me he encontrado un error 404 en la iteración {self.indice}")
                print(f"Aún así he podido sacar los datos de la película {self.nombre_peli} 🥷🏻")
                print(" ")
            
            except:
                self.info = 'ID no encontrada'
                print(f"Peli no encontrada en la iteración {self.indice}")
                print(" ")
            
        # Si encontró datos del reparto en el resumen de la info de la película, intenta extraerlos de la página
        if 'reparto' in self.info.lower():
            
            self.reparto_encontrado = True
            
            try:
                
                if 'reparto' in self.driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div[1]').text.lower():
                    
                    # Extrae los datos del reparto principal
                    self.datos_reparto = self.driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div[2]').text
                    
                    # Separa los datos obtenidos y se queda sólo con los nombres de los actores
                    self.datos_reparto = self.datos_reparto.split('\n')[::2] 
                    
                    # Si la longitud de la lista de actores obtenida es superior a 10, la limitamos 
                    if len(self.datos_reparto) > 10:
                        self.datos_reparto = self.datos_reparto[:10]
                
                elif 'reparto' in self.driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[1]').text.lower():
                    
                    # Extrae los datos del reparto principal
                    self.datos_reparto = self.driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]').text
                    
                    # Separa los datos obtenidos y se queda sólo con los nombres de los actores
                    self.datos_reparto = self.datos_reparto.split('\n')[::2] 
                    
                    # Si la longitud de la lista de actores obtenida es superior a 10, la limitamos 
                    if len(self.datos_reparto) > 10:
                        self.datos_reparto = self.datos_reparto[:10]  
                        
                elif 'reparto' in self.driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[1]').text.lower():
                    
                    # Extrae los datos del reparto principal
                    self.datos_reparto = self.driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]').text
                    
                    # Separa los datos obtenidos y se queda sólo con los nombres de los actores
                    self.datos_reparto = self.datos_reparto.split('\n')[::2] 
                    
                    # Si la longitud de la lista de actores obtenida es superior a 10, la limitamos 
                    if len(self.datos_reparto) > 10:
                        self.datos_reparto = self.datos_reparto[:10]    
            
            except:
                self.datos_reparto = '-'
    
        
    def extraccion_datos (self):
        
        """Extrae los datos de las películas desde IMDb."""
        
        self._preparacion()
        
        # Itera por cada indice y pelicula en cada fila del diccionario
        for self.indice, self.pelicula in self.peliculas_df.iterrows():

            self._busqueda_id() 
            
            self._obtencion_info() 
            
            # Almacena los datos encontrados sobre las películas 
            self.info_extraida[(self.nombre_peli , self.anio_peli)] = self.info
              
            # Almacena los datos encontrados sobre los actores
            if self.reparto_encontrado and self.datos_reparto != '-':
                self.lista_actores.extend(self.datos_reparto)
                self.lista_actores = list(set(self.lista_actores))
            
        # Cierra el navegador
        self.driver.close()
                    
        print(f"-- Extracción de datos completada con un total de {len(self.info_extraida)} elementos")
        print(f" ")
    
    
    def _limpieza_direccion_guion_argumento (self):
        
        """Realiza la limpieza de los datos relacionados con la dirección, guion y argumento de la película."""

        self.separador = re.compile(r'(?<=[a-z])(?=[A-Z])')
        
        if 'direccion' == unidecode(self.elemento.lower()):
            self.datos_direccion = self.info_separada[self.indice+1]
            
            # Utiliza el patrón de regex para separar los elementos de la string
            totales_d = self.separador.split(self.datos_direccion)
            
            self.datos_direccion = totales_d                  
        
        if 'guion' == unidecode(self.elemento.lower()):
            self.datos_guion = self.info_separada[self.indice+1]
            
            # Utiliza el patrón de regex para separar los elementos de la string
            totales_g = self.separador.split(self.datos_guion)
            
            self.datos_guion = totales_g
            
        if 'argumento' in self.elemento.lower() and unidecode(self.info_separada[self.indice+1].lower()) != 'direccion' and unidecode(self.info_separada[self.indice+1].lower()) != 'guion':
            self.argumento = self.info_separada[self.indice+1]
            
            
    def _limpieza_puntuacion_duracion (self):
        
        """Realiza la limpieza de los datos relacionados con la puntuación y la duración de la película."""
        
        self.patron_horas = r'\b(?:\d+h\s*)?\d+min\b'
        
        if 'puntuacion en imdb' == unidecode(self.elemento.lower()):
            self.puntuacion = self.info_separada[self.indice+1]
            self.puntuacion = float(self.puntuacion.replace(',' , '.'))
                                
        if re.search(self.patron_horas , self.elemento):
            self.horas = self.info_separada[self.indice]   
            
            try:
                # Si hay más de un elemento en la lista de duración, significa que también encontró horas, así que las convierte a minutos            
                if len(self.horas.split()) > 1:
                    self.duracion_minutos = int(self.horas.split()[0].replace('h' , ''))*60 + int(self.horas.split()[1].replace('min', ''))
            
                else:
                    self.duracion_minutos = int(self.horas.replace('min', '')) 
            except:
                    self.duracion_minutos = "-"
                
    
    def limpieza_datos (self):
        
        """Limpia los datos extraídos de las películas."""
        
        print(f"-- Empieza la limpieza de datos")
        print(f" ")
        
        self.lista_detalles_peliculas = []
                
        # Itera por la clave (tupla con nombre y año) y el valor (string de detalles) del diccionario de películas extraído
        for tupla , string in self.info_extraida.items(): 
            
            self.nombre_peli = tupla[0]
            self.anio_peli = tupla[1]
        
            if 'ID no encontrada' in string:
                tupla_peli = (np.nan , np.nan , np.nan , np.nan , np.nan , self.nombre_peli , self.anio_peli)
                
                # Añade la tupla al listado de películas
                self.lista_detalles_peliculas.append(tupla_peli)
                
            else:
                self.info_separada = string.split('\n')
                
                # Variables de inicio:
                self.puntuacion = "-"
                self.datos_direccion = "-"
                self.datos_guion = "-" 
                self.argumento = "-" 
                self.duracion_minutos ="-"
            
                # Una vez que la string se ha pasado a lista, itera por cada uno de sus elementos
                for self.indice , self.elemento in enumerate(self.info_separada):
                    
                    self._limpieza_direccion_guion_argumento()
                    self._limpieza_puntuacion_duracion() 
            
                # Crea una tupla con todos los detalles de la película
                tupla_peli = (self.puntuacion , self.datos_direccion , self.datos_guion , self.argumento , self.duracion_minutos , self.nombre_peli , self.anio_peli)
                
                # Añade la tupla al listado de películas
                self.lista_detalles_peliculas.append(tupla_peli)
                
        print(f"-- Limpieza de datos completada")
        print(f" ")
                
                
    def guardar_csv (self):
        
        """Guarda los datos extraídos en archivos CSV."""
        
        try:
            df_lista_detalles_peliculas = pd.DataFrame(self.lista_detalles_peliculas)
            df_lista_detalles_peliculas.to_csv(self.salida_detalles)
            print(f"Los detalles de las películas se han guardado en:")
            print(f"- {self.salida_detalles} ✅")
        except:
            print(f"Ha habido un error al intentar guardar el archivo con los detalles de las películas")
            print(f"Prueba a usar la variable 'detalles_pelis' para guardarlo de forma manual")
            print(" ") 
            pass
    
        try:
            df_lista_actores = pd.DataFrame(self.lista_actores)
            df_lista_actores.to_csv(self.salida_actores)
            print(f"Los detalles de las películas se han guardado en:")
            print(f"- {self.salida_actores} ✅")
        
        except:
            print(f"Ha habido un error al intentar guardar el archivo con los actores de las películas")
            print(f"Prueba a usar la variable 'lista_actores' para guardarlo de forma manual")
            print(" ") 
            pass
        
        
    def inicio (self):
        
        """
        Ejecuta el proceso completo de extracción, limpieza y guardado de datos.
        
        Retona:
            Una tupla que contiene las listas de detalles de películas y actores.
        """
        
        self.extraccion_datos()
        
        self.limpieza_datos()
        
        self.guardar_csv()
        
        return self.lista_detalles_peliculas , self.lista_actores
        

In [6]:
# Lista de variables a modificar según el archivo del que se deseen extraer los detalles:

ruta_csv_a_extraer = 'Proyectos\Modulo-2\Adalab-proyecto-da-promo-G-modulo-2-team-1\Drama\MovieDataBase\pelis_drama_2000_2004.csv' # Nombre o ruta del archivo del que queremos extraer información
columna_titulos = '1' # Nombre de la columna en la que se encuentran los títulos 
columna_anios = '2' # Nombre de la columna en la que se encuentran los años 
columna_id = '4' # Nombre de la columna en la que se encuentran las IDs 
ruta_salida_detalles_pelis = 'detalles_pelis_drama_2000_2004.csv' # Nombre o ruta del archivo en el que guardaremos los detalles 
ruta_salida_actores_pelis = 'actores_pelis_drama_2000_2004.csv' # Nombre o ruta del archivo en el que guardaremos los actores 

# Creacion de la clase:

extraccion = Obtencion_detalles_IMDb(ruta_csv_a_extraer, columna_titulos, columna_anios, columna_id, ruta_salida_detalles_pelis, ruta_salida_actores_pelis)

# Llamada a la función de inicio:

detalles_pelis , lista_actores = extraccion.inicio()


# SI LA FUNCIÓN DA ERROR: 
# Si los archivos no se encuentran en la misma ubicación que este jupyter, tendremos que darle la ruta. P.e: en lugar de 'pelis_drama_2000_2004.csv' >>> 'Drama/pelis_drama_2000_2004.csv'
# Hay que sustituir '1' y '2' por el nombre de las columnas en las que aparezca el título de la película y el año
# --- Por ejemplo: '1' >>> 'nombre' y '2' >>> 'aÃ±o estreno'
# --- Podemos saber el nombre de las columnas abriendo el archivo csv en un excel y mirando en la celda A1

FileNotFoundError: [Errno 2] No such file or directory: 'Proyectos\\Modulo-2\\Adalab-proyecto-da-promo-G-modulo-2-team-1\\Drama\\MovieDataBase\\pelis_drama_2000_2004.csv'